# Optimization of a Simplified Methane-Oxygen Rocket Engine Thrust Chamber
This notebook follows along with the report written for a class project: LINK

A simplified model of a regeneratively cooled rocket thrust chamber is optimized using constrained, gradient based optimization.


## Problem Statement and Design Variables
The problem at hand is to minimize the structural mass $m_{s}$ of a regeneratively cooled thrust chamber subject to thrust ($F$), efficiency ($I_{sp}$), mechanical stress ($\sigma$), and operating temperature $(T)$ inequality constraints given a cryogenic propellant combination.

Collecting all design variables in vector $\pmb{\mu}$, then this may be written mathematically as follows:
\begin{equation}
	\begin{split}
		\mathrm{minimize} \quad & m_{s}(\pmb{\mu}) \\
		\mathrm{subject \; to:} \quad & F_{min} - F(\pmb{\mu}) \leq 0\\
		& I_{sp,min} - I(\pmb{\mu}) \leq 0 \\
		& \| \sigma(\pmb{\mu}) \|_{\infty} - \sigma_{yield} \leq 0 \\
		& \|T(\pmb{\mu})\|_{\infty} - T_{max} \leq 0 \\
        \mathrm{given:} \quad & \big\{ \{Material \; properties\}, \{ Ambient \; conditions\}, \{Propellant \; combination\}  \big\}. \\
	\end{split}
\end{equation}
The design variables which populate $\pmb{\mu}$ are defined below.

| Design Variable   | Description                           | Units |
| :---:             | :---                                  | :---  |
|$r_{c}$            | Combustion chamber internal radius    | $m$   |
|$L_{c}$            | Combustion chamber length             |  $m$  |
|$\beta_{nc}$       | Nozzle converging section cone frustum half angle | deg. |
|$\beta_{nd}$       | Nozzle diverging section cone frustum half angle  | deg. |
|$\epsilon$         | Nozzle area ratio, $A_{e} / A_{t}$    | -     |
|$t_{w}$            | Internal structure wall thickness     | $m$   |
|$t_{j}$            | Cooling jacket channel thickness      | $m$   |
|$t_{s}$            | Outer shell thickness                 | $m$   |
|$\dot{m}_{f}$      | Fuel mass flow rate                   | $kg/s$ |
|$\phi$             | Equivalence ratio                     | -     |
|$p_{c}$            | Combustion chamber pressure           | atm   |
|$f_{c}$            | Fuel mass-flow coolant fraction, $\dot{m}_{f,cool}/\dot{m}_{f,total}$                     |   -    |

## Models and Analysis
To avoid getting bogged down and to get right into the interesting results, the bulk of model derivation and analysis will not be presented in detail here; the interested reader is directed to the report above.
Instead only a schematic diagram is introduced along with the design structure matrix, showing how the various sub-models are related.
### Geometry Specification
The geometry of the combustion chamber and nozzle is simplified and parameterized with the internal volume treated as a body of revolution divided into three sections: combustion chamber, converging nozzle section, and diverging nozzle section. 
All sections are surrounded by the cooling jacket channel and outer shell. 
A schematic of an engine cross-section is shown below where quantities in parentheses are computed from other design parameters. 

<img src="Figures/Engine_schematic.jpg" alt="Engine schematic" width="800" />

Analysis is carried out in cylindrical coordinates with the $z$ axis aligned with the engine axis, about which axisymmetry in all models and analysis is assumed. 
Positive $z$ is defined in the direction of propellant flow and is measured from the inside of the injector face at the top of the combustion chamber. 

### Design Structure Matrix
As implemented, the graph representing the overall engine model is acyclic, and thus each component may be solved successively, without iteration. 
The resulting design structure matrix is shown below, where the sub-models are solved in numberical order given a design $\pmb{\mu}$.
An 'x' above the shaded diagonal means the model on the diagonal depends on the output from the model to the left.
An 'x' below the shaded diagonal would represent a cyclic dependence, meaning the models would need to be coupled or iteratively converged.

<img src="Figures/dsm.png" alt="Design structure matrix" width="800"/>


In reality there is cyclic coupling between the heat transfer model and the flow model. 
However, this coupling is weak which is why it was not pursued for the simplified model here. 
Additionally, there is coupling between the heat transfer model and the thermochemical solver.
The coolant stream mixes with the main fuel flow and raises its temperature, augmenting the heat release and post combustion state. 
This effect is ignored in the current model implementation.
There is additional coupling between the stress model and the flow model as components under stress undergo displacement, altering the engine cross-section slightly. 
This would impact the flow but again, this is likely to be a weak coupling. 

## Gradient-Based Optimization Scheme
The scipy.optimize library is used with the SLSQP method. This is a Quasi-Newton method, 

# First Test: Unconstrained Optimization
First we will test that the optimizer is working correctly by performing unconstrained optimization.
In this scenario the algorithm should make the engine as small as possible given the limits on the design variables.

In [1]:
#import libraries and model functions from repo
import sys
import os
import pickle
import numpy as np
import scipy.optimize as sciopt

from dev_thermo import comp_thermochem
from dev_geom import comp_geom
from dev_mesh import comp_mesh
from dev_flow import comp_flow
from dev_heat import comp_heat
from dev_stress import comp_stress
from dev_mass import comp_mass
from dev_performance import comp_performance


In [2]:
def callback(mu):
    '''Callback function to track the design variables at each iteration '''
    global XHIST
    XHIST.append(mu)

It is a good idea to use normalized design variables to deal with the differences in gradient scaling and sensitivity among design variables.
EXPAND
The interface for each sub-model is a dictionary with named entries, while the optimizer uses a design variable vector.
Define a function which converts a vector of normalized design variables into a dictionary of dimensional and physically scaled quantities. 

The design variables are normalized using min-max scaling, that is a normalized quantity $\tilde{\mu_{i}}$ is computed from it's dimensional counterpart $\mu_{i}$ as,
$$
\tilde{\mu}_{i} = \frac{\mu_{i} - \mu_{i,min}}{\mu_{i,max} - \mu_{i,min}},
$$
where $\mu_{i,min}$/$\mu_{i,max}$ are the limits.
$\tilde{\mu}_{i} \in [0,1]$ provided $\mu_{i,min} \leq \mu_{i} \leq \mu_{i,max}$.

In [3]:
def design_vars_vec2dict(mu, limits):
    '''Convert the normalized input vector to a dictionary of scaled, dimensional values, given the provded limits
    
    Args:
        x (ndarray)     : vector of normalized (0-1) design variables
        limits (dict)   : named limit tuples, (minval, maxval) for each entry of the design variables, 
        to recover dimensional quantities
    Returns:
        phys_vals (dict)    : the design variables in dimensional quantities
    '''

    norm = {    'rc'        : mu[0],        # radius, combustion chamber [m]
                'Lc'        : mu[1],        # length, combustion chamber [m]
                'pc'        : mu[2],        # pressure, combustion chamber [atm]
                'beta_nc'   : mu[3],        # angle, converging nozzle [deg]
                'beta_nd'   : mu[4],        # angle, diverging nozzle [deg]
                'mdot_f'    : mu[6],        # fuel mass flow rate, [kg/s]
                'phi'       : mu[6],        # equivalence ratio CH4-O2 [-]
                'er'        : mu[7],        # nozzle expansion ratio
                'tw'        : mu[8],        # wall thickness
                'tc'        : mu[9],        # channel thickness
                'ts'        : mu[10],       # shell thickness
                'f_cool'    : mu[11]        # fraction of fuel to run through cooling loop
            }

    phys_vals = {}
    for k,v in norm.items():
        xmin, xmax = limits[k]
        phys_vals[k] = xmin + (v * (xmax - xmin))

    return phys_vals

Define a function to retrieve mesh settings, material properties, and boundary conditions, which are all treated as constant throughout the optimization.

In [4]:
def get_prop_param_bc():
    '''Get mesh settings, material properties, and thermodynamic boundary conditions'''

    mesh_param = {  'nr'        :   10,    
                    'nz_c'      :   50,
                    'nz_nc'     :   50,
                    'nz_t'      :   3,
                    'nz_nd'     :   100,
                }

    material_prop = {   'k_w'   : 45,       # thermal conductivity, [W/(m K)] wall
                        'k_s'   : 45,       # thermal conductivity, [W/(m K)] shell
                        'E_w'   : 200E+09,  # youngs modulus steel, [Pa]
                        'v_w'   : 0.29,     # poisson's ratio, [-] 
                        'cte_w' : 11e-06,       # coefficient of thermal expansion, [m/K]
                        's_yield' : 350E+06,    # steel yield strength, [Pa]
                        'rho'   : 8E+03         # steel density, [kg/m^3]
                    }

    flow_bc = { 'Tfuel'     : 200,   # Kelvin, boiling point of ~110K, pre combustion temperature
                'p_inf'      : 0.1,  # ambient pressure, [atm]
                'T_inf'     : 200,   # ambient temperature, [K]
                'rho_fuel'  : 260,   # methane fuel density, [kg/m^3] (liq 440, supercrit ~260)
                'k_fuel'    : 40e-03,   # methane thermal conductivity, liquid ~200e-03, supercritical ~40e-03-80e-03
                'Pr_fuel'   : 0.86,     # (0.86)fuel Prandtl number, estimate
                'mu_fuel'   : 15e-06,   # fuel dynamic viscosity - supercritical
                'h_inf'     : 10,       # ambient heat transfer coefficient
                'Tcool'     : 90        # coolant inlet temperature
                }
    cp_cool     = flow_bc['Pr_fuel'] * flow_bc['k_fuel'] / flow_bc['mu_fuel']
    flow_bc['cp_fuel'] = cp_cool
    return material_prop, mesh_param, flow_bc

Next define the objective function which computes the structural mass given a design. As seen in the DSM, the mass model depends only on the thermochemical and geometry models.
Also define a call to the full model.

In [5]:
def fobj_mass(mu, limits):
    '''Objective function - mass
    
    Args:
        x (ndarray)     : vector of normalized (0-1) design variables
        limits (dict)   : named limits, mu_min, mu_max for each entry of the design variables, 
                          to recover dimensional quantities

    Retuns:
        mass (float)    : the total structural mass
    '''

    design_vars = design_vars_vec2dict(mu, limits)

    material_prop, mesh_param, flow_bc = get_prop_param_bc()

    therm_out   = comp_thermochem(design_vars, flow_bc)

    geom_out    = comp_geom(design_vars, therm_out)

    mass_out    = comp_mass(design_vars, material_prop, geom_out)

    mass = mass_out['mtot']

    return mass

In [12]:
def f_model(mu, limits):
    '''Call the full model, given a vector of normalized inputs and a dictionary
    giving the physical bounds for each variable'''

    design_vars = design_vars_vec2dict(mu, limits)
    material_prop, mesh_param, flow_bc = get_prop_param_bc()

    therm_out   = comp_thermochem(design_vars, flow_bc)

    geom_out    = comp_geom(design_vars, therm_out)

    mesh_out    = comp_mesh(design_vars, geom_out, mesh_param)

    mass_out    = comp_mass(design_vars, material_prop, geom_out)

    flow_out    = comp_flow(design_vars, flow_bc, therm_out, geom_out, mesh_out, mesh_param)

    perf_out    = comp_performance(design_vars, flow_bc, geom_out, therm_out, flow_out)

    heat_out    = comp_heat(design_vars, flow_bc, material_prop, mesh_param,
                            therm_out, geom_out, mesh_out, flow_out)

    stress_out  = comp_stress(design_vars, mesh_param, material_prop, mesh_out, flow_out, heat_out)

    res = { 'therm_out'     : therm_out,
            'geom_out'      : geom_out,
            'mesh_out'      : mesh_out,
            'mass_out'      : mass_out,
            'flow_out'      : flow_out,
            'perf_out'      : perf_out,
            'heat_out'      : heat_out,
            'stress_out'    : stress_out,
            'material_prop' : material_prop,
            'mesh_param'    : mesh_param,
            'flow_bc'       : flow_bc,
            }

    return res

Now we are ready to define a function to run unconstrained optimization.

In [10]:
def uncontrained_opitmization():
    global XHIST
    XHIST = []

    #define initial design - right in the middle of limits
    n_design_vars   = 12
    x0              = np.ones(n_design_vars) * 0.5

    # all design variables bounded between 0-1
    bounds = [ (0, 1),]*n_design_vars
    
    limits = {  'rc'        : (0.1, 0.5),           # radius, combustion chamber [m]
                'Lc'        : (0.1, 0.4),           # length, combustion chamber [m]
                'pc'        : (5, 40),              # pressure, combustion chamber [atm]
                'beta_nc'   : (30, 60),             # angle, converging nozzle [deg]
                'beta_nd'   : (15, 25),             # angle, diverging nozzle [deg]
                'mdot_f'    : (10, 11),             # fuel mass flow rate, [kg/s]
                'phi'       : (0.8, 1.2),           # equivalence ratio CH4-O2 [-]
                'er'        : (2, 30),              # nozzle expansion ratio
                'tw'        : (0.005, 0.05),        # wall thickness
                'tc'        : (0.001, 0.05),        # channel thickness
                'ts'        : (0.001, 0.0015),      # shell thickness
                'f_cool'    : (0.01, 0.5)           # fraction of fuel to run through cooling loop
            }

    # set up args, run optimizer
    args = (limits,)
    fobj = fobj_mass
    res = sciopt.minimize(  fun     = fobj, 
                            x0      = x0, 
                            args    = args,
                            method  = 'SLSQP',
                            bounds  = bounds, 
                            callback = callback)

    # extract optimal design and model output
    mu_opt_norm     = res['x']
    model_out       = f_model(mu_opt_norm, limits)
    mu_opt_dim      = design_vars_vec2dict(mu_opt_norm, limits)

    data = {    'model_out'     : model_out,
                'mu_opt_dim'    : mu_opt_dim,
                'opt_res'       : res,
                'x0'            : x0,
                'bounds'        : bounds,
                'limits'        : limits,
                'xhist'         : XHIST.copy()
            }
    return data

In [24]:
unc_output = uncontrained_opitmization()

In [42]:
def print_design_summary(opt_output):
    mu_opt = opt_output['mu_opt_dim']
    print('Engine size and performance:')
    print('*----------------------------------------------*')
    print('Total mass [kg]: {:1.3e}'.format(opt_output['model_out']['mass_out']['mtot'])) 
    print('Ideal engine thrust [N] : {:1.3e}'.format(opt_output['model_out']['perf_out']['F_ideal']))
    print('Engine thrust [N] : {:1.3e}'.format(opt_output['model_out']['perf_out']['F']))
    print('Ideal Isp [s] : {:1.3e}'.format(opt_output['model_out']['perf_out']['Isp_ideal']))
    print('Isp [s] : {:1.3e}'.format(opt_output['model_out']['perf_out']['Isp']))
    print('Ideal thrust coefficient [-] : {:1.3e}'.format(opt_output['model_out']['perf_out']['Cf_ideal']))
    print('Thrust coefficient [-] : {:1.3e}'.format(opt_output['model_out']['perf_out']['Cf']))
    print('Characteristic velocity (c*) [m/s] : {:1.3e}'.format(opt_output['model_out']['perf_out']['c_star']))

    print('*----------------------------------------------*')
    print('Engine design variables:')
    print('*----------------------------------------------*')

    for k,v in unc_output['limits'].items():
        print('{}: {:1.3e}'.format(k, mu_opt[k]))
    print('*----------------------------------------------*')


In [44]:
print_design_summary(unc_output)

Engine size and performance:
*----------------------------------------------*
Total mass [kg]: 4.691e+00
Ideal engine thrust [N] : 1.293e+05
Engine thrust [N] : 1.243e+05
Ideal Isp [s] : 2.770e+02
Isp [s] : 2.663e+02
Ideal thrust coefficient [-] : 1.495e+00
Thrust coefficient [-] : 1.437e+00
Characteristic velocity (c*) [m/s] : 1.818e+03
*----------------------------------------------*
Engine design variables:
*----------------------------------------------*
rc: 1.000e-01
Lc: 1.000e-01
pc: 4.000e+01
beta_nc: 6.000e+01
beta_nd: 2.500e+01
mdot_f: 1.100e+01
phi: 1.200e+00
er: 2.000e+00
tw: 5.000e-03
tc: 2.550e-02
ts: 1.250e-03
f_cool: 2.550e-01
*----------------------------------------------*


The unconstrained optimization has performed as expected, with the chamber radius (rc) and length (Lc) reaching the lower end of the bounds, the nozzle converging (beta_nc) and diverging (beta_nd) sections contracting/expanding as quickly as possible to the smallest expansion ratio (er).

# Isp-only constraint
Next we introduce a constraint on the engine efficiency; Isp. Define a function which computes this constraint.

In [45]:
def constr_Isp(x, limits, Ispmin):
    '''Isp constraint function'''

    design_vars = design_vars_vec2dict(x, limits)
    material_prop, mesh_param, flow_bc = get_prop_param_bc()

    therm_out   = comp_thermochem(design_vars, flow_bc)

    geom_out    = comp_geom(design_vars, therm_out)

    mesh_out    = comp_mesh(design_vars, geom_out, mesh_param)

    flow_out    = comp_flow(design_vars, flow_bc, therm_out, geom_out, mesh_out, mesh_param)

    perf_out    = comp_performance(design_vars, flow_bc, geom_out, therm_out, flow_out)

    Isp = perf_out['Isp']
    val = Isp - Ispmin
    return val

In [46]:
def isp_constrained_opitmization(Ispmin=None):
    assert Ispmin is not None
    global XHIST
    XHIST = []

    #define initial design - right in the middle of limits
    n_design_vars   = 12
    x0              = np.ones(n_design_vars) * 0.5

    # all design variables bounded between 0-1
    bounds = [ (0, 1),]*n_design_vars
    
    limits = {  'rc'        : (0.1, 0.5),           # radius, combustion chamber [m]
                'Lc'        : (0.1, 0.4),           # length, combustion chamber [m]
                'pc'        : (5, 40),              # pressure, combustion chamber [atm]
                'beta_nc'   : (30, 60),             # angle, converging nozzle [deg]
                'beta_nd'   : (15, 25),             # angle, diverging nozzle [deg]
                'mdot_f'    : (10, 11),             # fuel mass flow rate, [kg/s]
                'phi'       : (0.8, 1.2),           # equivalence ratio CH4-O2 [-]
                'er'        : (2, 30),              # nozzle expansion ratio
                'tw'        : (0.005, 0.05),        # wall thickness
                'tc'        : (0.001, 0.05),        # channel thickness
                'ts'        : (0.001, 0.0015),      # shell thickness
                'f_cool'    : (0.01, 0.5)           # fraction of fuel to run through cooling loop
            }

    # set up the constraint
    cIsp = {    'type'  : 'ineq',
                'fun'   : constr_Isp,
                'args'  : (limits, Ispmin,)
            }
    constraints = [cIsp,]
    
    # set up args, run optimizer
    args = (limits,)
    fobj = fobj_mass
    res = sciopt.minimize(  fun         = fobj, 
                            x0          = x0, 
                            args        = args,
                            method      = 'SLSQP',
                            bounds      = bounds, 
                            constraints = constraints,
                            callback    = callback)

    # extract optimal design and model output
    mu_opt_norm     = res['x']
    model_out       = f_model(mu_opt_norm, limits)
    mu_opt_dim      = design_vars_vec2dict(mu_opt_norm, limits)

    data = {    'model_out'     : model_out,
                'mu_opt_dim'    : mu_opt_dim,
                'opt_res'       : res,
                'x0'            : x0,
                'bounds'        : bounds,
                'limits'        : limits,
                'xhist'         : XHIST.copy()
            }
    return data

Now run the optimization with a constraint of $Isp_{min}=290s$.

In [47]:
isp290_output = isp_constrained_opitmization(Ispmin=290)

In [48]:
print_design_summary(isp290_output)

Engine size and performance:
*----------------------------------------------*
Total mass [kg]: 8.778e+00
Ideal engine thrust [N] : 1.408e+05
Engine thrust [N] : 1.353e+05
Ideal Isp [s] : 3.018e+02
Isp [s] : 2.900e+02
Ideal thrust coefficient [-] : 1.629e+00
Thrust coefficient [-] : 1.565e+00
Characteristic velocity (c*) [m/s] : 1.818e+03
*----------------------------------------------*
Engine design variables:
*----------------------------------------------*
rc: 1.000e-01
Lc: 1.000e-01
pc: 4.000e+01
beta_nc: 6.000e+01
beta_nd: 2.417e+01
mdot_f: 1.100e+01
phi: 1.200e+00
er: 4.075e+00
tw: 5.000e-03
tc: 2.550e-02
ts: 1.250e-03
f_cool: 2.550e-01
*----------------------------------------------*
